In [1]:
%load_ext autoreload
%autoreload 2
%reload_ext autoreload

In [2]:
import pandas as pd
import numpy as np
%matplotlib inline
import matplotlib
import matplotlib.pyplot as plt
import math as mt
import warnings

# 한글출력
matplotlib.rc('font', family='AppleGothic')
plt.rcParams['axes.unicode_minus'] = False
warnings.filterwarnings(action='ignore') 

from src import crs, PublicPredictor
from src.dbc import utils, TimeDivisionKMeans

In [ ]:
# 1. load_excel
data_path = "data/apt_1.xlsx"
xlsx = pd.read_excel(data_path, header=None,
                     skiprows=2, engine="openpyxl")

# 2. data preprocessing
p, m = crs.utils.data_preprocessing(xlsx)

m.set_index("month", inplace=True)

# 3. data init
_month = 1

month_df = pd.DataFrame(m.loc[_month])
month_df.reset_index(inplace=True)

month_df.columns = ['name', 'usage (kWh)']

PUBLIC_PERCENTAGE = 30
APT = crs.utils.get_APT(month_df, PUBLIC_PERCENTAGE)

calc = crs.models.ManagementOffice(
        month=_month,
        households=month_df,
        APT=APT,
        contract="단일계약"
    )
apt = calc.apart

In [ ]:
fee = calc.fee
public_bill = calc.public_bill
public_bill

In [ ]:
_public_bill = public_bill / len(month_df)
_public_bill

# 데이터 준비

In [ ]:
m_15 = utils.data_preprocessing(xlsx)

df = utils.dimension_reduction(m_15)
m_60 = df.copy()

m_60.head()

In [ ]:
# 1월치 데이터 파싱
m_15_1 = m_15[m_15.index.month == 1].copy()
m_15_1.head()

In [ ]:
m_60_1 = m_60[m_60.index.month == 1].copy()
m_60_1.head()

In [ ]:
m_60_1[m_60_1.columns[:5]].head()

# 클러스터링 셋팅
- TimeDivisionKMeans 다시 개발

In [ ]:
from src.TimeDivisionKMeans import TimeDivisionKMeans

tdKMeans = TimeDivisionKMeans(datas=m_60_1.T.values)

In [ ]:
tdKMeans.fit()

In [ ]:
plt.figure(figsize=(16,8))

plt.plot(tdKMeans.datas.T, color='g', linewidth=0.4)
plt.plot(tdKMeans.clusters_.T, color='b', linewidth=0.4)

plt.show()

# 기여도 산정

## 그룹화

### label number 부여

In [ ]:
_month_df = np.column_stack([m_60_1.columns, m_60_1.sum(axis=0).values.round().astype("int")])
month_df = pd.DataFrame(_month_df, columns=['name', 'usage (kWh)'])

month_df

In [ ]:
labels, clusters = tdKMeans.labels_, tdKMeans.clusters_

_cluster_info = np.column_stack([month_df.values, labels])
cluster_info = pd.DataFrame(_cluster_info, columns=['name', 'usage (kWh)','그룹 번호'])

cluster_info.head(10)

In [ ]:
_labels = np.unique(labels)
label_cnt = np.array([np.where(labels == _label)[0].size for _label in _labels])

_cont = np.column_stack([_labels, clusters.sum(axis=1).round().astype("int"),label_cnt])
cont = pd.DataFrame(_cont, columns=['그룹 번호', 'median (kWh)','가구 수'])
cont.set_index("그룹 번호", inplace=True)

cont

## 기여도 산정 1. 그룹의 중간값 ( 클러스터 패턴의 사용량 ) 을 이용한 기여도 산정

In [ ]:
cont_1 = cont.copy()

total_median = cont_1['median (kWh)'].sum()
cont_1['contribution'] = cont_1['median (kWh)'].map(lambda x: x / total_median)

cont_1

## 기여도 산정 2. 그룹 안에 속해있는 가구들의 사용량 총합으로 기여도 산정

In [ ]:
cont_2 = cont.copy()

households_kWh = cluster_info['usage (kWh)'].sum()

cont_2['usage total'] = np.array(
    [cluster_info[cluster_info['그룹 번호'] == _label]['usage (kWh)'].sum() for _label in _labels]
)
cont_2['contribution'] = cont_2['usage total'].map(lambda x: x / households_kWh)

cont_2

# 가격분배

## 단순 가격 분배

In [ ]:
_public_bill = round(public_bill / len(cluster_info))
_public_bill

In [ ]:
simple_dist_1 = cont_1.copy()

simple_dist_1['공용부 요금'] = simple_dist_1['가구 수'].map(lambda x: x * _public_bill)
simple_dist_1['단순 가격분배'] = (simple_dist_1['공용부 요금'] * simple_dist_1['contribution']).round().astype("int")

simple_dist_1['가구 별 분배 가격'] = simple_dist_1.apply(lambda x: int(x['단순 가격분배'] / x['가구 수']), axis=1)

simple_dist_1

In [ ]:
simple_info_1 = cluster_info.copy()

simple_info_1['단순 가격분배'] = simple_info_1['그룹 번호'].map(
        lambda x: int(simple_dist_1.loc[x]['가구 별 분배 가격'])
)

simple_info_1['공용부 요금 나머지'] = round((public_bill - simple_info_1['단순 가격분배'].sum()) / len(cluster_info))
simple_info_1['최종 공용부 요금'] = simple_info_1['단순 가격분배'] + simple_info_1['공용부 요금 나머지']

simple_info_1.head(10)

In [ ]:
plt.figure(figsize=(16,8))

ax_1 = plt.subplot(2,1,1)
ax_1.bar(range(len(simple_dist_1)), simple_dist_1['contribution'], color='b', label='기여도')
ax_1.set_title("그룹별 기여도 현황")
ax_1.legend()

ax_2 = plt.subplot(2,1,2)
ax_2.bar(range(len(simple_info_1)), simple_info_1['최종 공용부 요금'], color='g', label='가구별 공용부요금')
ax_2.set_title("가구별 가격분배 현황")
ax_2.legend()

plt.show()

In [ ]:
min_h = simple_info_1[simple_info_1['name'] == '아파트1-104-905']
mean_h = simple_info_1[simple_info_1['name'] == '아파트1-102-801']
max_h = simple_info_1[simple_info_1['name'] == '아파트1-102-903']

problem_simple_cont = pd.concat([min_h, mean_h, max_h])
problem_simple_cont

## 기여도 요금

In [ ]:
# 예측 시뮬레이션
pp = PublicPredictor(
    APT=APT,
    month_df=month_df[['name', 'usage (kWh)']],
    month=_month
)
pp.predict

In [ ]:
col_df = cont.copy()

col_df['공용부 기본요금'] = (round(pp.basic / len(month_df)) * col_df['가구 수']).round().astype("int")
col_df['공용부 전력량요금'] = round(pp.elec_rate / len(month_df)) * col_df['가구 수'].round().astype("int")

col_df

In [ ]:
dist_1 = pd.concat([cont_1, col_df[col_df.columns[-2:]]], axis=1)

dist_1['기여도 공용부 기본요금'] = (dist_1['공용부 기본요금'] * dist_1['contribution']).round().astype("int")
dist_1['기여도 공용부 전력량요금'] = (dist_1['공용부 전력량요금'] * dist_1['contribution']).round().astype("int")
dist_1['기여도 공용부 요금'] = dist_1['기여도 공용부 기본요금'] + dist_1['기여도 공용부 전력량요금']
dist_1['가구 별 분배 가격'] = (dist_1['기여도 공용부 요금'] / dist_1['가구 수']).round().astype("int")

dist_1

In [ ]:
dist_info_1 = cluster_info.copy()

dist_info_1['단순 가격분배'] = dist_info_1['그룹 번호'].map(
        lambda x: int(dist_1.loc[x]['가구 별 분배 가격'])
)

dist_info_1['공용부 요금 나머지'] = round((public_bill - dist_info_1['단순 가격분배'].sum()) / len(dist_info_1))
dist_info_1['최종 공용부 요금'] = dist_info_1['단순 가격분배'] + dist_info_1['공용부 요금 나머지']

dist_info_1.head(10)

In [ ]:
plt.figure(figsize=(16,8))

ax_1 = plt.subplot(2,1,1)
ax_1.bar(range(len(dist_1)), dist_1['contribution'], color='b', label='기여도')
ax_1.set_title("그룹별 기여도 현황")
ax_1.legend()

ax_2 = plt.subplot(2,1,2)
ax_2.bar(range(len(dist_info_1)), dist_info_1['최종 공용부 요금'], color='g', label='가구별 공용부요금')
ax_2.set_title("가구별 가격분배 현황")
ax_2.legend()

plt.show()

In [ ]:
min_h = dist_info_1[dist_info_1['name'] == '아파트1-104-905']
mean_h = dist_info_1[dist_info_1['name'] == '아파트1-102-801']
max_h = dist_info_1[dist_info_1['name'] == '아파트1-102-903']

problem_simple_cont = pd.concat([min_h, mean_h, max_h])
problem_simple_cont

## 기여도 적용비율 요금

In [ ]:
norm_dist = cont_1.copy()

norm_dist['contribution'] = (norm_dist['contribution'] - norm_dist['contribution'].min()) / \
                    (norm_dist['contribution'].max() - norm_dist['contribution'].min())

norm_dist = pd.concat([norm_dist, col_df[col_df.columns[-2:]]], axis=1)

norm_dist['기여도 공용부 기본요금'] = (norm_dist['공용부 기본요금'] * norm_dist['contribution']).round().astype("int")
norm_dist['기여도 공용부 전력량요금'] = (norm_dist['공용부 전력량요금'] * norm_dist['contribution']).round().astype("int")
norm_dist['기여도 공용부 요금'] = norm_dist['기여도 공용부 기본요금'] + norm_dist['기여도 공용부 전력량요금']
norm_dist['가구 별 분배 가격'] = (norm_dist['기여도 공용부 요금'] / norm_dist['가구 수']).round().astype("int")

norm_dist

In [ ]:
norm_dist_info = cluster_info.copy()

norm_dist_info['단순 가격분배'] = norm_dist_info['그룹 번호'].map(
        lambda x: int(norm_dist.loc[x]['가구 별 분배 가격'])
)

norm_dist_info['공용부 요금 나머지'] = round((public_bill - norm_dist_info['단순 가격분배'].sum()) / len(norm_dist_info))
norm_dist_info['최종 공용부 요금'] = norm_dist_info['단순 가격분배'] + norm_dist_info['공용부 요금 나머지']

norm_dist_info.head(10)

In [ ]:
plt.figure(figsize=(16,8))

ax_1 = plt.subplot(2,1,1)
ax_1.bar(range(len(dist_1)), norm_dist['contribution'], color='b', label='기여도')
ax_1.set_title("그룹별 기여도 현황")
ax_1.legend()

ax_2 = plt.subplot(2,1,2)
ax_2.bar(range(len(dist_info_1)), norm_dist_info['최종 공용부 요금'], color='g', label='가구별 공용부요금')
ax_2.set_title("가구별 가격분배 현황")
ax_2.legend()

plt.show()

In [ ]:
min_h = norm_dist_info[norm_dist_info['name'] == '아파트1-104-905']
mean_h = norm_dist_info[norm_dist_info['name'] == '아파트1-102-801']
max_h = norm_dist_info[norm_dist_info['name'] == '아파트1-102-903']

problem_simple_cont = pd.concat([min_h, mean_h, max_h])
problem_simple_cont

## 피드백 활용